<a href="https://colab.research.google.com/github/Nicocoding/Revolutsheet/blob/main/Revolut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
#!pip install ipdb

# Install es_ES
!/usr/share/locales/install-language-pack es_ES
!dpkg-reconfigure locales

# Restart Python process to pick up the new locales
import os
os.kill(os.getpid(), 9)



In [ ]:
from requests import check_compatibility
import pandas as pd 
import yfinance as yf 
import locale

from google.colab import auth
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

auth.authenticate_user()

import gspread 
from google.auth import default 
creds, _ = default()
gc = gspread.authorize(creds)

sh=gc.open('Revolut_Statement').sheet1
rows =sh.get_all_values()
dfrev = pd.DataFrame(rows)
New_Header = dfrev.iloc[0]
dfrev = dfrev[1:]
dfrev.columns = New_Header.values
dfrev['Date'] = pd.to_datetime(dfrev['Date'])
dfrev['Date'] = dfrev['Date'].dt.date

sh=gc.open('Logfile')
wks = sh.worksheet('Logfile')
rows = wks.get_all_values()
dflog = pd.DataFrame(rows)
New_Header = dflog.iloc[0]
dflog = dflog[1:]
dflog.columns = New_Header.values
dflog['Date'] = pd.to_datetime(dflog['Date'], dayfirst = True)
dflog['Date'] = dflog['Date'].dt.date

chop = dflog.iloc[-1,0]
dfrev = dfrev[~(dfrev['Date'] < chop)]
dfrev = dfrev[dfrev.Type != 'CASH TOP-UP']
dfrev = dfrev[dfrev.Type != 'CUSTODY FEE']
dfrev.reset_index(inplace=True, drop=True)

wks = sh.worksheet('Sheet1')
wks2 = sh.worksheet('Portfolio')
rows2 =wks2.get_all_values()
portfolio = pd.DataFrame(rows2)
New_Header = portfolio.iloc[0]
portfolio = portfolio[1:]
portfolio.columns = New_Header.values

wks2 = sh.worksheet('Taxes')
rows2 =wks2.get_all_values()
taxes = pd.DataFrame(rows2)
New_Header = taxes.iloc[0]
taxes = taxes[1:]
taxes.columns = New_Header.values

for index, rows in dfrev.iterrows():

  #Date
  date = rows['Date']
  datemod = date.strftime("%d/%m/%Y")
  wks.update_cell(index + 2, 1, datemod)

  #Ticker
  ticker = rows['Ticker']
  wks.update_cell(index + 2, 2, ticker)

  #Short Name
  tick = yf.Ticker(ticker)
  name = tick.info['shortName']
  wks.update_cell(index + 2, 3, name)

  #Quote type
  quoteType = tick.info['quoteType']
  wks.update_cell(index + 2, 4, quoteType)

  #Type of transaction
  typeTRX = rows['Type']
  typeTRX = typeTRX.split()[0]
  wks.update_cell(index + 2, 5, typeTRX)

  #Currency
  ccy = tick.info['currency']
  wks.update_cell(index + 2, 6, ccy)

  #Exchange rate with respect to EUR
  tickccy = "EUR" + ccy + "=X"
  tickccy = yf.Ticker(tickccy)
  hist = tickccy.history(start=date, end=date)
  hist = hist["Close"].iloc[0]
  hist = round(hist,4)
  wks.update_cell(index + 2, 7, hist)

  #Quantity
  if typeTRX == 'DIVIDEND': 
    portfolupd = portfolio.loc[portfolio['Ticker'] == ticker]
    portfolupd = portfolupd.astype({'Qty':'float'})
    quantity = portfolupd['Qty'].sum()
    wks.update_cell(index + 2, 8, quantity)
  else:
    quantity = locale.atof(rows['Quantity'])
    wks.update_cell(index + 2, 8, quantity)

  #Price
  if typeTRX == 'DIVIDEND':
    total = locale.atof(rows['Total Amount'].strip('$'))
    total = total/hist
    price = total / quantity
  else:
    price = locale.atof(rows['Price per share'].strip('$'))
    price = price/hist
  wks.update_cell(index + 2, 9, price)

  #Fees
  total = locale.atof(rows['Total Amount'].strip('$'))
  total = total/hist
  if typeTRX == 'BUY':
    fees = total - (quantity*price)
    total = total -fees
  if typeTRX == 'SELL':
    fees = (quantity*price) - total
  if typeTRX == 'DIVIDEND':
    fees = 0
  wks.update_cell(index + 2, 10, fees)
  wks.update_cell(index + 2, 11, total)

  #Country
  country = tick.info['country']
  countryupd = taxes.loc[taxes['Country'] == country]
  if countryupd.empty: 
    tax = "19%"
  else:
    tax = countryupd.iloc[0]["Final tax rate"]
  wks.update_cell(index + 2, 12, tax)
  wks.update_cell(index + 2, 13, country)

  #Sector
  sector = tick.info['sector']
  wks.update_cell(index + 2, 14, sector)
  
  #Industry
  industry = tick.info['industry']
  wks.update_cell(index + 2, 15, industry)